<a href="https://colab.research.google.com/github/SnipeHunting/MQ135-Air-Quality-with-ST7735-Display/blob/master/ModernPortfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
#import sys
#!{sys.executable} -m pip install yfinance --upgrade --no-cache-dir

#!{sys.executable} -m pip install fix_yahoo_finance

#!{sys.executable} -m pip install pandas_datareader --upgrade

#!{sys.executable} -m pip install numpy --upgrade

#!{sys.executable} -m pip install pybind11

#!{sys.executable} -m pip install riskfolio-lib

import pandas as pd

import numpy as np
import scipy as sc
from scipy import optimize
import datetime as dt
from pandas_datareader import data as pdr
import yfinance as yf
import plotly.graph_objects as go
import sklearn.metrics as metrics
import riskfolio as rp
yf.pdr_override()

In [8]:
# Import data
def getData(stocks, start, end):
    stockData = pdr.get_data_yahoo(stocks, start=start, end=end)
    stockData = stockData['Close']

    returns = stockData.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return meanReturns, covMatrix


stockList = ['CVX', 'AOS', 'MCD', 'INTC', 'F', 'MSFT']
stocks = [stock+'' for stock in stockList] # for none US stocks

endDate = dt.datetime.now()
startDate = endDate-dt.timedelta(days=800)


In [9]:
def portfolioPerformance(weights, meanReturns, covMatrix):
    returns = np.sum(meanReturns*weights)*252
    std = np.sqrt(
            np.dot(weights.T,np.dot(covMatrix, weights))
           )*np.sqrt(252) # W transpose * sigma of W
    return returns, std

In [10]:
def negativeSR(weights, meanReturns, covMatrix, riskFreeRate = 0.05):  #negative sharp ratio.   Need to update the riskFreeInrerest
    pReturns, pStd = portfolioPerformance(weights, meanReturns, covMatrix)
    return - (pReturns - riskFreeRate)/pStd  # will give the negative sharp ratio

def maxSR(meanReturns, covMatrix, riskFreeRate = 0.05, constraintSet=(0,1)):  #constraint for now is between 0 and 1
    "Minimize the negative SR, by altering the weights of the portfolio"
    numAssets = len(meanReturns) # checks the number of stocks 
    args = (meanReturns, covMatrix, riskFreeRate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1}) # checking that all the summation are equal to 1
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.optimize.minimize(negativeSR, numAssets*[1./numAssets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints) # sequantial least square .  First guess is making all weigths equal
    return result

#  weights = np.array([0.3,0.3,0.4])

meanReturns, covMatrix = getData(stocks,start=startDate, end=endDate)
#  returns, std = portfolioPerformance(weights, meanReturns,covMatrix)

result = maxSR(meanReturns, covMatrix)
#maxSR,maxWeights = result['fun'], result['x']

#print(maxSR,maxWeights) # maximize sharp ratio by changing the assets allocations (trade off between return and volatility)

[*********************100%***********************]  6 of 6 completed


#Minimium Portfolio Variance

In [11]:
def portfolioVariance(weights, meanReturns, covMatrix):
    return portfolioPerformance(weights, meanReturns, covMatrix)[1]

def minimizeVariance(meanReturns, covMatrix, constraintSet=(0,1)):
    """Minimize the portfolio variance by altering the 
     weights/allocation of assets in the portfolio"""
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    result = sc.optimize.minimize(portfolioVariance, numAssets*[1./numAssets], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

minVarResult = minimizeVariance(meanReturns, covMatrix)
minVar,minVarWeights = minVarResult['fun'], minVarResult['x']
print(minVar,minVarWeights) # provides the allocation for minimum variablity portfolio

0.1590823586989816 [0.08663498 0.19121254 0.         0.         0.66515985 0.05699263]


#Creating the Efficient Frontier

In [12]:
def portfolioReturn(weights, meanReturns, covMatrix):
        return portfolioPerformance(weights, meanReturns, covMatrix)[0]


def efficientOpt(meanReturns, covMatrix, returnTarget, constraintSet=(0,1)):
    """For each returnTarget, we want to optimise the portfolio for min variance"""
    numAssets = len(meanReturns)
    args = (meanReturns, covMatrix)

    constraints = ({'type':'eq', 'fun': lambda x: portfolioReturn(x, meanReturns, covMatrix) - returnTarget},
                    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = constraintSet
    bounds = tuple(bound for asset in range(numAssets))
    effOpt = sc.optimize.minimize(portfolioVariance, numAssets*[1./numAssets], args=args, method = 'SLSQP', bounds=bounds, constraints=constraints)
    return effOpt

##Use a return function that could be called by a graphing package like Plotly/Dash combination.

In [13]:
def calculatedResults(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0,1)):
    """Read in mean, cov matrix, and other financial information
        Output, Max SR , Min Volatility, efficient frontier """
    # Max Sharpe Ratio Portfolio
    maxSR_Portfolio = maxSR(meanReturns, covMatrix)
    maxSR_returns, maxSR_std = portfolioPerformance(maxSR_Portfolio['x'], meanReturns, covMatrix)

    maxSR_allocation = pd.DataFrame(maxSR_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    maxSR_allocation.allocation = [round(i*100,0) for i in maxSR_allocation.allocation] # to round decimal values
    
    # Min Volatility Portfolio
    minVol_Portfolio = minimizeVariance(meanReturns, covMatrix)
    minVol_returns, minVol_std = portfolioPerformance(minVol_Portfolio['x'], meanReturns, covMatrix)

    minVol_allocation = pd.DataFrame(minVol_Portfolio['x'], index=meanReturns.index, columns=['allocation'])
    minVol_allocation.allocation = [round(i*100,0) for i in minVol_allocation.allocation]

    # Efficient Frontier
    efficientList = []
    targetReturns = np.linspace(minVol_returns, maxSR_returns, 20) # define the number of points in the curve
    for target in targetReturns:
        efficientList.append(efficientOpt(meanReturns, covMatrix, target)['fun'])

    maxSR_returns, maxSR_std = round(maxSR_returns*100,2), round(maxSR_std*100,2)
    minVol_returns, minVol_std = round(minVol_returns*100,2), round(minVol_std*100,2)

    return maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns


print(calculatedResults(meanReturns, covMatrix))


(29.01, 24.14,       allocation
AOS          0.0
CVX         72.0
F           10.0
INTC         0.0
MCD         18.0
MSFT         0.0, 15.2, 15.91,       allocation
AOS          9.0
CVX         19.0
F            0.0
INTC         0.0
MCD         67.0
MSFT         6.0, [0.1590825332337113, 0.15937497172487655, 0.16025329606715075, 0.16170852863190657, 0.1637250342875601, 0.16628243845379415, 0.16935063200693853, 0.17283337973184, 0.17668672227165907, 0.18090580036441414, 0.18550961409670044, 0.19049910412560214, 0.19585829237984148, 0.20155698774197883, 0.20756778415380012, 0.2138641020441032, 0.22042157481780306, 0.2272175591838528, 0.23423132612385514, 0.2414438598868374], array([0.15203976, 0.15930656, 0.16657336, 0.17384016, 0.18110696,
       0.18837376, 0.19564056, 0.20290736, 0.21017416, 0.21744096,
       0.22470776, 0.23197456, 0.23924136, 0.24650816, 0.25377496,
       0.26104176, 0.26830856, 0.27557535, 0.28284215, 0.29010895]))


#Visualising the Efficient Frontier

In [14]:
def EF_graph(meanReturns, covMatrix, riskFreeRate=0, constraintSet=(0,1)):
    """Return a graph ploting the min vol, max sr and efficient frontier"""
    maxSR_returns, maxSR_std, maxSR_allocation, minVol_returns, minVol_std, minVol_allocation, efficientList, targetReturns = calculatedResults(meanReturns, covMatrix, riskFreeRate, constraintSet)

    #Max SR
    MaxSharpeRatio = go.Scatter(
        name='Maximium Sharpe Ratio',
        mode='markers',
        x=[maxSR_std],
        y=[maxSR_returns],
        marker=dict(color='red',size=14,line=dict(width=3, color='black'))
    )

    #Min Vol
    MinVol = go.Scatter(
        name='Mininium Volatility',
        mode='markers',
        x=[minVol_std],
        y=[minVol_returns],
        marker=dict(color='green',size=14,line=dict(width=3, color='black'))
    )

    #Efficient Frontier
    EF_curve = go.Scatter(
        name='Efficient Frontier',
        mode='lines',
        x=[round(ef_std*100, 2) for ef_std in efficientList],
        y=[round(target*100, 2) for target in targetReturns],
        line=dict(color='black', width=4, dash='dashdot')
    )

    data = [MaxSharpeRatio, MinVol, EF_curve]

    layout = go.Layout(
        title = 'Portfolio Optimisation with the Efficient Frontier',
        yaxis = dict(title='Annualised Return (%)'),
        xaxis = dict(title='Annualised Volatility (%)'),
        showlegend = True,
        legend = dict(
            x = 0.75, y = 0, traceorder='normal',
            bgcolor='#E2E2E2',
            bordercolor='black',
            borderwidth=2),
        width=800,
        height=600)
    
    fig = go.Figure(data=data, layout=layout)
    return fig.show()

In [15]:
EF_graph(meanReturns, covMatrix)

In [16]:
print(efficientOpt(meanReturns, covMatrix,0.225))  

     fun: 0.18570276980041292
     jac: array([0.11280485, 0.25846516, 0.22767229, 0.14439357, 0.10952542,
       0.11004565])
 message: 'Optimization terminated successfully'
    nfev: 56
     nit: 8
    njev: 8
  status: 0
 success: True
       x: array([2.62820258e-17, 4.79169411e-01, 4.07119036e-02, 0.00000000e+00,
       4.80118685e-01, 1.60461922e-17])


In [17]:
print(efficientOpt(meanReturns, covMatrix,0.24))  

     fun: 0.19643772086908173
     jac: array([0.11138236, 0.26922313, 0.23320969, 0.14197257, 0.09673519,
       0.10489843])
 message: 'Optimization terminated successfully'
    nfev: 42
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([0.00000000e+00, 5.34344525e-01, 5.52084502e-02, 1.94949063e-17,
       4.10447025e-01, 2.08166817e-17])


In [18]:
def plot_confusion_matrix(y_true, y_pred, class_names):
    confusion_matrix = metrics.confusion_matrix(y_true, y_pred)
    confusion_matrix = confusion_matrix.astype(int)

    layout = {
        "title": "Confusion Matrix", 
        "xaxis": {"title": "Predicted value"}, 
        "yaxis": {"title": "Real value"}
    }

    fig = go.Figure(data=go.Heatmap(z=confusion_matrix,
                                    x=class_names,
                                    y=class_names,
                                    hoverongaps=False),
                    layout=layout)
    fig.show()